In [1]:
# default_exp wp2md

# wp2md

> Convert Wordpress to Markdown Files

In [2]:
#export
from fastcore.utils import urljson, AttrDict, Path, first, test_eq
from fastcore.script import call_parse
from IPython.display import Markdown
from markdownify import markdownify as md

In [3]:
#hide
from nbdev.showdoc import *

In [4]:
#export
def _getpost(post_id:int, baseurl:str):
    return AttrDict(urljson(f'{baseurl}/{post_id}'))

In [5]:
_post = _getpost(247, 'https://outerbounds.com/wp-json/wp/v2/posts')
assert _post

In [6]:
#export
class WP:
    def __init__(self, id:int, baseurl:str):
        self.post = _getpost(id, baseurl=baseurl)
        
    _props = ['title', 'date', 'tags', 'draft', 'description', 'image', 'slug']
              
    @property
    def draft(self) -> str: 
        return str(self.post.draft != 'publish').lower()
    
    @property
    def title(self) -> str:
        title = self.post.get('title', None)
        return title.get('rendered', None) if title else title
    
    @property
    def image(self) -> str:
        img = self.post.get('uagb_featured_image_src', None)
        return first(img.get('large', [])) if img else img
    
    @property
    def frontmatter(self) -> str:
        fm = '---\n'
        for p in self._props:
            attr = getattr(self, p, None)
            if attr: fm+=f'{p}: "{attr}"\n'
        return fm+'---\n'
    
    def __getattr__(self, name):
        return self.post.get(name, None)
    
    @property
    def markdown(self) -> str: 
        "Return the markdown representation of the body of the post."
        return self.frontmatter + md(self.post.content['rendered'])

    def tomd(self, dest_path:str=None, dest_file:str=None) -> None:
        "Write markdown representation of wordpress post"
        if not dest_path: dest_path = '.'
        if not dest_file: dest_file = self.slug+'.md'
        p = Path(dest_path)/dest_file
        print(f'Writing: {p}')
        p.write_text(self.markdown)

Instantiating a `WP` object will allow you to get access to useful properties that can render as front matter.  The postid can be extracted from wordpress edit url, for example the id for `https://outerbounds.com/wp-admin/post.php?post=220&action=edit` is `220`.

In [7]:
_post = WP(id=220, baseurl='https://outerbounds.com/wp-json/wp/v2/posts')
print(_post.frontmatter)

---
title: "Notebooks In Production With Metaflow"
date: "2022-02-09T22:59:06"
image: "https://outerbounds.com/wp-content/uploads/2022/02/Screen-Shot-2022-02-09-at-12.45.20-pm-1024x525.png"
slug: "notebooks-in-production-with-metaflow"
---



In [8]:
_result="""---
title: "Notebooks In Production With Metaflow"
date: "2022-02-09T22:59:06"
image: "https://outerbounds.com/wp-content/uploads/2022/02/Screen-Shot-2022-02-09-at-12.45.20-pm-1024x525.png"
slug: "notebooks-in-production-with-metaflow"
---
"""
test_eq(_post.frontmatter, _result)

The markdown representation is also available as a property (below we print the first 1,000 characters:

In [9]:
print(_post.markdown[:1200])

---
title: "Notebooks In Production With Metaflow"
date: "2022-02-09T22:59:06"
image: "https://outerbounds.com/wp-content/uploads/2022/02/Screen-Shot-2022-02-09-at-12.45.20-pm-1024x525.png"
slug: "notebooks-in-production-with-metaflow"
---


By Hamel Husain


*Learn how to use notebooks in production ML workflows with a new Metaflow feature*


When building production-ready machine learning systems, it is critical to monitor the health and performance of those systems with reports and visualizations. Furthermore, allowing for rapid debugging and interactive introspection is critical when workflows fail or do unexpected things. Jupyter notebooks have often been a preferred tool of data scientists for these tasks of visualization, exploration, debugging, and rapid iteration.  Ironically, many production systems do not integrate appropriately with notebooks, which can significantly frustrate progress on these tasks.


Indeed, in the field of data science tooling, one of the most [hotly-co

## CLI Utility For Wordpress To Markdown

In [10]:
#export
@call_parse
def wp2md(id:int, 
          baseurl:str='https://outerbounds.com/wp-json/wp/v2/posts', # the baseurl that corresponds to the wp api
          dest_path:str='.', # The path to save the markdown file to
          dest_file:str=None # Optional, if not given defaults to the slug indicated in wordpress
         ):
    "Convert A wordpress post into markdown file with front matter."
    post = WP(id=id, baseurl=baseurl)
    post.tomd(dest_path=dest_path, dest_file=dest_file)